In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



img_path = "dataset/lung_colon_image_set/colon_image_sets"

transform = transforms.Compose([
    transforms.ToTensor(),          # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

dataset = datasets.ImageFolder(root=img_path, transform=transform)

print(dataset.class_to_idx)



{'colon_aca': 0, 'colon_n': 1}


In [10]:
from torch.utils.data import random_split

split_precentage = 0.8
train_size = int(split_precentage * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [11]:
import torch.nn as nn
import torch.nn.functional as F

class ModelCNN(nn.Module):
    def __init__(self):
        super(ModelCNN, self).__init__()
        # Define CNN layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers (size dynamically inferred)
        self.fc1 = nn.Linear(64 * 192 * 192, 128)  # Adjusted based on earlier calculations
        self.fc2 = nn.Linear(128, 1)  # Output layer for binary classification

    def forward(self, x):
        # Pass through the layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically flatten the output from convolutional layers
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch

        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [12]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ModelCNN()
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
import time
num_epochs = 10

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        # Move inputs and labels to GPU if available
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    end_time = time.time()  # End time for the epoch
    epoch_duration = end_time - start_time  # Calculate the duration
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Time: {epoch_duration:.2f} seconds")

KeyboardInterrupt: 

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to GPU if available
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")